In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
ref_form = r"""方程组：
                解：$\left\{\begin{array}{l}5 x+4 y=6,① \\ 2 x+3 y=1 ;②\end{array}\right.$
由$①×2$，得：$10x + 8y = 12.③$ 
由$②×5$，得： $10x + 15y = 5.④$
$③-④$，得： $-7y = 7,$
解得：$y=-1$.
把$y=-1$代入$①$，得：$5x-4=6$,
解得：$x=2$.
"""
template = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ###板书一:     ###讲解一:       ###板书二:       ###讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。###板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$###讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。###板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$###讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
"""

In [5]:
llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)
prompt = ChatPromptTemplate.from_template(template)
parser = StrOutputParser()

chain = prompt | llm | parser

In [6]:
import json
with open("test_problem_set.json", "r") as file:
    data = json.load(file)

In [8]:
problems = []
answers = []
for ps in data:
    problems.append(ps['problem'])
    answers.append(ps['answer'])

In [21]:
final_notes = []
final_expl = []

In [22]:
import re
log = []
for problem in problems:
    print("processing: \n" + problem + '\n')
    output = chain.invoke({"question": problem, "ref_form": ref_form})
    log.append(output)
    # 提取板书内容
    notes = re.findall(r'(###板书[一二三]:\n\([^\)]+\)\n[^\n]+)', output)
    all_notes = "\n".join(notes)

    # 提取讲解内容
    explanations = re.findall(r'###讲解[一二三]:\n((?:[^\n]+\n)+)', output)
    all_explanations = "\n".join(explanations)

    final_notes.append(all_notes)
    final_expl.append(all_explanations)

processing: 
把下列各数填入表示它所在的数集的大括号中：

-2.4, π, 2.022, -10/3, -0.15, 0, -√9, -10, -1.101 001 000 1⋯。

(1) 整数集合：{ … }；

(2) 正实数集合：{ … }；

(3) 无理数集合：{ … }。

processing: 
已知关于x的不等式a(x-1) > x + 1 - 2a的解集是x < -1，求a的取值范围。

processing: 
7. 定义新运算：对于任意实数a, b(其中a≠0)，都有$a ⊗ b = \frac{1}{a} - \frac{a - b}{a}$，等式右边是通常的加法、减法及除法运算，比如：$2 ⊗ 1 = \frac{1}{2} - \frac{2 - 1}{2} = 0$。

(1) 3 ⊗ 4的值为______；
(2) 若2 ⊗ x的值小于1的数，求x的非负整数解。

processing: 
在甲处劳动的工人有35人，在乙处劳动的工人有19人。现在另调21人去支援，使得在甲处的人数为乙处人数的3/2倍，应调往甲、乙两处各多少人？

processing: 
某工地派96人去挖土和运土，如果平均每人每天挖土5 m³或运土3 m³，那么该怎样分配挖土和运土的人数，才能使挖出的土刚好及时运走？

processing: 
已知关于x的方程$$\frac{2x - m}{3} = \frac{2 - x}{3}$$的解是非负数，若$$m$$是正整数，求$$m$$的值。

processing: 
如果(m+3)x > 2m + 6的解集为x < 2，那么m的取值范围是  
A. m < 0  
B. m < -3  
C. m > -3  
D. m是任意实数

processing: 
在等式 \( y=ax^2+bx+c \) 中，当 \( x=1 \) 时，\( y=2 \)；当 \( x=-1 \) 时，\( y=-2 \)；当 \( x=2 \) 时，\( y=3 \)。则 \( a= \_\_\_\_\_\_\) ，\( b= \_\_\_\_\_\_\)，\( c= \_\_\_\_\_\_\)。

processing: 
3. 求下列各式的值：
(1) √196 = ；
(2) √(16/169) = ；
(

In [24]:
write2 = []
for i in range(len(problems)):
    cur = {}
    cur['problem'] = problems[i]
    # cur['notes'] = final_notes[i]
    # cur['explanation'] = final_expl[i]
    cur['answer'] = log[i]
    write2.append(cur)

with open("outputs/model_0_output.json", "w", encoding="utf-8") as file:
    json.dump(write2, file, ensure_ascii=False, indent=4)

In [13]:
import re

text = """
知识点讲解:本题考查的知识点是数集的分类，解题关键点是根据数的性质将其分类到不同的数集中。

###板书一:
(1)
整数集合：$\{0, -\sqrt{9}, -10\}$

###板书二:
(2)
正实数集合：$\{2.022\}$

###板书三:
(3)
无理数集合：$\{\pi, -1.1010010001\ldots\}$

###讲解一:
首先，我们需要判断哪些数属于整数。在题目中，整数集合包含正整数、负整数和零。通过观察数列，可以确定整数有0、$-\sqrt{9}$（因为$-\sqrt{9}=-3$，是整数）和-10。

###讲解二:
其次，我们要找出正实数集合。正实数是所有大于零的数，包括有理数和无理数。在题目中，唯一一个正实数是2.022。

###讲解三:
最后，我们要确定哪些数属于无理数。无理数是不能表示为两个整数之比的小数，其中包括π和-1.1010010001…。因此，这两个数属于无理数集合。

故答案是：
(1)整数集合：$\{0, -\sqrt{9}, -10\}$；
(2)正实数集合：$\{2.022\}$；
(3)无理数集合：$\{\pi, -1.1010010001\ldots\}$。
"""

# 提取板书内容
board_content = re.findall(r'###板书[一二三]:\n\((\d)\)\n([^\n]+)', text)
for number, content in board_content:
    print(f'板书{number}: {content}')

# 提取讲解内容
explanation_content = re.findall(r'###讲解[一二三]:\n([^\n]+\n)+', text)
for i, content in enumerate(explanation_content, 1):
    print(f'讲解{i}: {content.strip()}')


板书1: 整数集合：$\{0, -\sqrt{9}, -10\}$
板书2: 正实数集合：$\{2.022\}$
板书3: 无理数集合：$\{\pi, -1.1010010001\ldots\}$
讲解1: 首先，我们需要判断哪些数属于整数。在题目中，整数集合包含正整数、负整数和零。通过观察数列，可以确定整数有0、$-\sqrt{9}$（因为$-\sqrt{9}=-3$，是整数）和-10。
讲解2: 其次，我们要找出正实数集合。正实数是所有大于零的数，包括有理数和无理数。在题目中，唯一一个正实数是2.022。
讲解3: 最后，我们要确定哪些数属于无理数。无理数是不能表示为两个整数之比的小数，其中包括π和-1.1010010001…。因此，这两个数属于无理数集合。


In [15]:
import re

text = """
知识点讲解:本题考查的知识点是数集的分类，解题关键点是根据数的性质将其分类到不同的数集中。

###板书一:
(1)
整数集合：$\{0, -\sqrt{9}, -10\}$

###板书二:
(2)
正实数集合：$\{2.022\}$

###板书三:
(3)
无理数集合：$\{\pi, -1.1010010001\ldots\}$

###讲解一:
首先，我们需要判断哪些数属于整数。在题目中，整数集合包含正整数、负整数和零。通过观察数列，可以确定整数有0、$-\sqrt{9}$（因为$-\sqrt{9}=-3$，是整数）和-10。

###讲解二:
其次，我们要找出正实数集合。正实数是所有大于零的数，包括有理数和无理数。在题目中，唯一一个正实数是2.022。

###讲解三:
最后，我们要确定哪些数属于无理数。无理数是不能表示为两个整数之比的小数，其中包括π和-1.1010010001…。因此，这两个数属于无理数集合。

故答案是：
(1)整数集合：$\{0, -\sqrt{9}, -10\}$；
(2)正实数集合：$\{2.022\}$；
(3)无理数集合：$\{\pi, -1.1010010001\ldots\}$。
"""

# 提取板书内容
notes = re.findall(r'(###板书[一二三]:\n\([^\)]+\)\n[^\n]+)', text)
all_notes = "\n".join(notes)

# 提取讲解内容
explanations = re.findall(r'###讲解[一二三]:\n((?:[^\n]+\n)+)', text)
all_explanations = "\n".join(explanations)

print(f'Note:\n{all_notes}')
print(f'Explanation:\n{all_explanations}')


Note:
###板书一:
(1)
整数集合：$\{0, -\sqrt{9}, -10\}$
###板书二:
(2)
正实数集合：$\{2.022\}$
###板书三:
(3)
无理数集合：$\{\pi, -1.1010010001\ldots\}$
Explanation:
首先，我们需要判断哪些数属于整数。在题目中，整数集合包含正整数、负整数和零。通过观察数列，可以确定整数有0、$-\sqrt{9}$（因为$-\sqrt{9}=-3$，是整数）和-10。

其次，我们要找出正实数集合。正实数是所有大于零的数，包括有理数和无理数。在题目中，唯一一个正实数是2.022。

最后，我们要确定哪些数属于无理数。无理数是不能表示为两个整数之比的小数，其中包括π和-1.1010010001…。因此，这两个数属于无理数集合。

